In [0]:
#extract data from drive into colab local context
!tar -xf "/content/drive/My Drive/repo_data/ssd-object-detection/data/ShelfImages.tar"
# pull the files from my github
!git clone https://github.com/arpytanshu/ssd-object-detection

Cloning into 'ssd-object-detection'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 124 (delta 70), reused 95 (delta 41), pack-reused 0
Receiving objects: 100% (124/124), 117.27 KiB | 11.73 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [0]:
cd ssd-object-detection/

/content/ssd-object-detection


In [0]:
import utils
import torch
from ssdconfig import SSDConfig
from data import ShelfImageDataset, collate_fn, get_dataframe
from torch.utils.data import DataLoader
from torch.optim import SGD
from ssd import SSD, MultiBoxLoss
from trainer import train, eval

config = SSDConfig()
device = config.DEVICE

config.PATH_TO_ANNOTATIONS = "/content/ssd-object-detection/data/annotation.txt"
config.PATH_TO_IMAGES = "/content/ShelfImages/"
config.PATH_TO_CHECKPOINT = "/content/drive/My Drive/repo_data/ssd-object-detection/checkpoint/checkpoint_ssd_1.pth.tar"

config.PRINT_FREQ = 35
config.VGG_BN_FLAG = True
config.TRAIN_BATCH_SIZE = 8
config.LEARNING_RATE = 0.001
config.USE_PRETRAINED_VGG = False
config.NUM_ITERATIONS_TRAIN = 4000 


In [0]:
# dataloader
df = get_dataframe(config.PATH_TO_ANNOTATIONS)
dataset_tr = ShelfImageDataset(df, config.PATH_TO_IMAGES, train=True)
dataloader_tr = DataLoader(dataset_tr,
                           shuffle=True,
                           collate_fn=collate_fn,
                           batch_size=config.TRAIN_BATCH_SIZE,
                           num_workers=config.NUM_DATALOADER_WORKERS)

dataset_te = ShelfImageDataset(df, config.PATH_TO_IMAGES, train=False)
dataloader_te = DataLoader(dataset_te,
                           shuffle=True,
                           collate_fn=collate_fn,
                           batch_size=config.TRAIN_BATCH_SIZE,
                           num_workers=config.NUM_DATALOADER_WORKERS)


In [0]:
try:
    checkpoint = torch.load(config.PATH_TO_CHECKPOINT)
    start_epoch = checkpoint['epoch'] + 1
    print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']
except FileNotFoundError:
    print('PATH_TO_CHECKPOINT not specified in SSDConfig.\nMaking new model and optimizer.')
    start_epoch = 0
    model = SSD(config)
    model_parameters = utils.get_model_params(model)
    optimizer = SGD(params=[{'params': model_parameters['biases'], 'lr': 2 * config.LEARNING_RATE},
                        {'params': model_parameters['not_biases']}],
                        lr=config.LEARNING_RATE,
                        momentum=config.MOMENTUM,
                        weight_decay=config.WEIGHT_DECAY)

# move to device
model.to(device)
criterion = MultiBoxLoss(model.priors_cxcy, config).to(device)
# num epochs to train
epochs = config.NUM_ITERATIONS_TRAIN // len(dataloader_tr)
# epoch where LR is decayed
decay_at_epoch = [int(epochs*x) for x in config.DECAY_LR_AT]


# fooh!!!! :)
for epoch in range(start_epoch, epochs):
    if epoch in decay_at_epoch:
        utils.adjust_learning_rate(optimizer, config.DECAY_FRAC)
    train(dataloader_tr, model, criterion, optimizer, epoch)
    if (epoch%5 == 0):
      print('Model checkpoint.', end=' ' )
      utils.save_checkpoint(epoch, model, optimizer, config, config.PATH_TO_CHECKPOINT)
      print('Model Evaluation.', end=' :: ')
      print('mAP: ' + str(eval(model, dataloader_te, 0.6, 0.4)))

utils.save_checkpoint(epoch, model, optimizer, config, config.PATH_TO_CHECKPOINT)

PATH_TO_CHECKPOINT not specified in SSDConfig.
Making new model and optimizer.


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Epoch: [0][0/36]	Batch Time 2.167 (2.167)	Data Time 1.799 (1.799)	Loss 4.4698 (4.4698)	
Epoch: [0][35/36]	Batch Time 0.091 (0.981)	Data Time 0.001 (0.760)	Loss nan (nan)	
Model checkpoint. Model Evaluation. :: mAP: 0.0
Epoch: [1][0/36]	Batch Time 2.034 (2.034)	Data Time 1.657 (1.657)	Loss 3.7571 (3.7571)	
Epoch: [1][35/36]	Batch Time 0.168 (0.971)	Data Time 0.077 (0.750)	Loss 3.2891 (3.6634)	
Epoch: [2][0/36]	Batch Time 2.536 (2.536)	Data Time 2.302 (2.302)	Loss 3.5514 (3.5514)	
Epoch: [2][35/36]	Batch Time 0.091 (0.995)	Data Time 0.001 (0.779)	Loss 3.3271 (3.4629)	
Epoch: [3][0/36]	Batch Time 2.212 (2.212)	Data Time 1.984 (1.984)	Loss 3.4625 (3.4625)	
Epoch: [3][35/36]	Batch Time 0.092 (1.003)	Data Time 0.001 (0.789)	Loss 3.4030 (3.3946)	
Epoch: [4][0/36]	Batch Time 2.211 (2.211)	Data Time 1.980 (1.980)	Loss 3.2836 (3.2836)	
Epoch: [4][35/36]	Batch Time 0.091 (0.956)	Data Time 0.001 (0.741)	Loss 3.4475 (3.2864)	
Epoch: [5][0/36]	Batch Time 2.308 (2.308)	Data Time 2.093 (2.093)	Loss 3.